# Softmax_Regression_with_Tensorflow
* by:陈炳炎(bjchenbingyan@126.com)
* blog: http://www.intelligentdata.cn/


先前我们实现一个简易的回归模型，该回归模型输出的值是连续值，因此回归模型适用于预测数值型数据。但是现实中，很多问题是分类问题，我们可以通过对回归模型的进一步升级为logsistic回归，使其适用于分类问题。但是传统的logistic回归模型适用于二分类问题，为了满足多分类问题的要求，可以将logistic回归进一步推广到适用于多分类问题的softmax回归。

具体办法让回归模型有多个输出（该输出数目与类别数相同），然后接入一层softmax层进行交叉熵计算，从而改变输出的概率分布。通过比较该分布的输出来确定模型预测的具体类别。
softmax函数：
![softmax function](softmax_regression_img/softmax.jpg)

本项目利用softmax regression算法对机器学习经典的手写数字数据集MNIST进行分类。
MNIST由数万张28×28像素的手写数字组成，这些图片只包含灰度值信息。我们的任务是将这些图片分成0-9十个类别。

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
# 使用tensorflow中的input_data模块导入数据
from tensorflow.examples.tutorials.mnist import input_data

## 1.导入数据，探索数据结构

In [2]:
# 导入数据，如果指定路径没有数据，tensorflow会自动下载，但由于网络原因经常无法下载成功，因此可以自己下载后放到该路径下
# label为独热编码（onehot encoding）
mnist = input_data.read_data_sets('./data/mnist_data', one_hot=True)

Extracting ./data/mnist_data\train-images-idx3-ubyte.gz
Extracting ./data/mnist_data\train-labels-idx1-ubyte.gz
Extracting ./data/mnist_data\t10k-images-idx3-ubyte.gz
Extracting ./data/mnist_data\t10k-labels-idx1-ubyte.gz


In [3]:
# 打印MNIST数据集的相关信息
print('Training data shape:\t', mnist.train.images.shape)
print('Training labels shape:\t', mnist.train.labels.shape)
print('Validation data shape:\t', mnist.validation.images.shape)
print('Validation labels shape:\t', mnist.validation.labels.shape)
print('Testing data shape:\t', mnist.test.images.shape)
print('Testing labels shape:\t', mnist.test.labels.shape)

Training data shape:	 (55000, 784)
Training labels shape:	 (55000, 10)
Validation data shape:	 (5000, 784)
Validation labels shape:	 (5000, 10)
Testing data shape:	 (10000, 784)
Testing labels shape:	 (10000, 10)


## 2. 定义计算公式损失函数

定义计算公式，即按照softmax regression的训练步骤定义各步的数据。
对于损失函数，一般选用交叉熵来评估多分类问题的模型好坏。cross-entropy的公式如下：
![cross-entropy](softmax_regression_img/cross-entropy.jpg)

In [5]:
# 定义两个placeholder分别用于保存image和label信息：None表示输入图片数量未知，784表示一张图片有784像素
x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float', [None, 10])
# 定于W和b分别用于保存模型权重，其中W的维度是[784, 10]（因为一张图片有784列），b的维度是[10, 1]
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
# softmax regression model
y_pred = tf.nn.softmax(tf.matmul(x, W) + b)
# cost function: cross entropy（交叉熵）
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_pred), reduction_indices=1))
# 定义optimiizer：使用梯度下降法进行优化
learning_rate = 0.1
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

In [6]:
# prediction
# argmax返回tensor中最大值的序列，1代表列
prediction = tf.equal(tf.argmax(y_pred, 1), tf.arg_max(y, 1))
# accuracy
# 将bool值转为float32的数据，以便计算准确率
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
# initializer
init = tf.global_variables_initializer()

## 对数据进行迭代训练

In [12]:
epochs = 500
batch_size = 100
# Session
sess = tf.Session()
sess.run(init)

# 迭代500个epochs
# 对每个epoch，按每100个样本为一个batch_size进行迭代训练
for epoch in range(epochs):
    avg_cost= 0.
    # 计算batch数量
    num_batch = int(mnist.train.num_examples / batch_size)
    for i in range(num_batch):
        # 获取每个batch的image和label
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feeds = {x: batch_xs, y: batch_ys}
        sess.run(optimizer, feed_dict = feeds)
        avg_cost += sess.run(cross_entropy, feed_dict=feeds) / num_batch
    if epoch % 10 == 0:
        train_feeds = {x: batch_xs, y: batch_ys}
        test_feeds = {x: mnist.test.images, y: mnist.test.labels}
        train_accuracy = sess.run(accuracy, feed_dict= train_feeds)
        test_accuracy = sess.run(accuracy, feed_dict= test_feeds)
        print('Epoch: %03d/%03d cost: %.9f train_acc: %.3f test_acc: %.3f' % (epoch, epochs, avg_cost, train_accuracy, test_accuracy))

Epoch: 000/500 cost: 0.526277160 train_acc: 0.920 test_acc: 0.903
Epoch: 010/500 cost: 0.276526231 train_acc: 0.880 test_acc: 0.921
Epoch: 020/500 cost: 0.259532402 train_acc: 0.980 test_acc: 0.923
Epoch: 030/500 cost: 0.251236627 train_acc: 0.930 test_acc: 0.924
Epoch: 040/500 cost: 0.245885644 train_acc: 0.900 test_acc: 0.924
Epoch: 050/500 cost: 0.240467963 train_acc: 0.910 test_acc: 0.924
Epoch: 060/500 cost: 0.240310103 train_acc: 0.960 test_acc: 0.925
Epoch: 070/500 cost: 0.237225443 train_acc: 0.940 test_acc: 0.925
Epoch: 080/500 cost: 0.238478543 train_acc: 0.930 test_acc: 0.925
Epoch: 090/500 cost: 0.236743390 train_acc: 0.930 test_acc: 0.926
Epoch: 100/500 cost: 0.231117153 train_acc: 0.970 test_acc: 0.926
Epoch: 110/500 cost: 0.233455260 train_acc: 0.930 test_acc: 0.927
Epoch: 120/500 cost: 0.232208499 train_acc: 0.960 test_acc: 0.926
Epoch: 130/500 cost: 0.230553993 train_acc: 0.930 test_acc: 0.925
Epoch: 140/500 cost: 0.230113124 train_acc: 0.940 test_acc: 0.926
Epoch: 150